In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [47]:
path_global = '/Users/bdu/Library/CloudStorage/OneDrive-SharedLibraries-epfl.ch/ENAC-IT - Pilot_iAQ_Shared/Data/'

# meta data

In [118]:
df_meta = pd.read_excel(path_global+'Raw/0001_scholair/generic/240412_SCOLAIR_donnees_batiments.xlsx')[['Ref','Campagne_str','Localite_str','Point_mesure','SCOLAIR_Plus','Ventilation','Context','Built_year','Mes_g_d_pose','Mes_g_d_retr']]
df_meta['Ref'] = df_meta['Ref'].astype(str)

df_meta['SCOLAIR_Plus'] = df_meta['SCOLAIR_Plus'].str.replace('SCO','W1')
df_meta['SCOLAIR_Plus'] = df_meta['SCOLAIR_Plus'].str.replace('SP1','W1')
df_meta['SCOLAIR_Plus'] = df_meta['SCOLAIR_Plus'].str.replace('SP2','W2')
df_meta['SCOLAIR_Plus'] = df_meta['SCOLAIR_Plus'].str.replace('SP3','W3')
df_meta['Period identifier'] = df_meta['Campagne_str']+df_meta['SCOLAIR_Plus']

df_meta['Loc_number'] = df_meta['Ref'].apply(lambda x: x[2:4])
df_meta_subset = df_meta[['Ref','Loc_number','Point_mesure','Period identifier']].drop_duplicates().sort_values(by='Loc_number')
df_meta_subset.rename(columns = {'Loc_number':'Building identifier','Point_mesure':'Room identifier'},inplace=True)
df_meta_subset

,Ref,Building identifier,Room identifier,Period identifier
0,11010104,01,EXT,A21W1
197,13010304,01,NHA,E22W1
99,12010304,01,NHA,H22W1
98,12010204,01,NBA,H22W1
97,12010104,01,EXT,H22W1
...,...,...,...,...
96,11240304,24,NHA,A21W1
95,11240204,24,NBA,A21W1
94,11240104,24,EXT,A21W1
290,13240304,24,NHA,E22W1


# COVA data

In [128]:
import glob
coa = pd.DataFrame() #storing cleaned data
path = path_global+'Raw/0001_scholair/COA_cova/' #where data from this sensor is stored
all_files = glob.glob(path + '*.csv')
for file in all_files:
    identifier = file.split('_')[6].split('.')[0] #extract identifier, eg: '../data/raw/COA_cova/data_COA_11050204.csv'
    df = pd.read_csv(file, index_col=0)
    df.columns = ['COA_CAS','COA_Name_list','COA_Name_file','COA_R.time','ppb','ug/m3','COA_Probability'] #remove identifer from column names
    df = df[['COA_Name_list','ppb','ug/m3']].set_index(['COA_Name_list'])
    df = df.transpose()
    df['Ref'] = identifier
    df['Unit'] = df.index
    df = df.set_index('Ref')
    coa = pd.concat([coa, df])
    coa = coa.sort_values(by='Ref')

In [129]:
coa = coa.rename_axis(None, axis=1).reset_index()
coa.rename(columns = {'Total(As Toluene)':'TVOC'},inplace=True)
coa = coa.merge(df_meta_subset, how='left',on='Ref')
coa = coa.drop('Ref',axis=1)
coa['equipment'] = 'kit-cova'
coa.to_csv('0001_scholair_cova.csv')

# Tenax data

In [131]:
import glob
ten = pd.DataFrame()
path = path_global+'Raw/0001_scholair/COA_ten/'
all_files = glob.glob(path + '*.xlsx')
for file in all_files:
    df = pd.read_excel(file, index_col=0,converters={'Ref':str})
    df = df[df['Ref']!='blanc']
    df = df.dropna(subset=['Ref'])
    ten = pd.concat([ten, df])

In [132]:
ten['equipment'] = 'Tenax'
ten['unit'] = 'ug/m3'
ten = ten.rename(columns={'COVtot':'TVOC'})
ten = ten.merge(df_meta_subset, how='left',on='Ref')
ten = ten.drop(['N_unique','Site','Date','Composes_doses','Ref'],axis=1) #drop irrelevant columns
ten.to_csv('0001_scholair_tenax.csv') #I still can't edit Onedrive locally, so I uploaded the output files using browser